# Створення музики за допомогою Magenta

[Magenta](https://magenta.tensorflow.org/) бібліотека для мови програмування Python для генерації музики та образотворчого мистецтва.

# Рекурентні нейронні мережі (RNN)

![Розгорнута мережа](images/rnn_unrolled.png)

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

# Локальні налаштування

Завантаження бібліотек:

In [ ]:
import sys
!{sys.executable} -m pip install magenta==2.1.3 pretty-midi==0.2.9 pyFluidSynth==1.3.0 tensorflow==2.3.1

In [ ]:
import note_seq

from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle

from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

from note_seq.sequences_lib import repeat_sequence_to_duration

Допоміжні функції:

In [ ]:
def play(seq):
    note_seq.plot_sequence(seq)
    note_seq.play_sequence(seq, synth=note_seq.fluidsynth)
    
def compose(input_melody, rnn, temperature):
    start_time = input_melody.total_time
    end_time = 2 * input_melody.total_time

    generator_options = generator_pb2.GeneratorOptions()
    generator_options.args['temperature'].float_value = temperature
    generator_options.generate_sections.add(start_time=start_time, end_time=end_time)
    
    result = rnn.generate(input_melody, generator_options)
    
    return result

def merge_from(seq1, seq2=None):

    result=music_pb2.NoteSequence()
    result.ticks_per_quarter = seq1.ticks_per_quarter

    for t in seq1.tempos:
        result.tempos.add(qpm=t.qpm)

    for ks in seq1.key_signatures:
        result.key_signatures.add(key=ks.key)

    for ts in seq1.time_signatures:
        result.time_signatures.add(numerator=ts.numerator,
                                   denominator=ts.denominator)

    max_instrument_seq1 = 0
    for note in seq1.notes:
        instrument = note.instrument if note.instrument else 1
            
        if instrument > max_instrument_seq1:
            max_instrument_seq1 = instrument
        
        result.notes.add(pitch=note.pitch,
                         velocity=note.velocity,
                         start_time=note.start_time,
                         end_time=note.end_time,
                         program=note.program,
                         instrument=instrument,
                         is_drum=note.is_drum)

    if seq2:
        for note in seq2.notes:
            result.notes.add(pitch=note.pitch,
                             velocity=note.velocity,
                             start_time=note.start_time,
                             end_time=note.end_time,
                             program=note.program,
                             instrument=note.instrument + max_instrument_seq1,
                             is_drum=note.is_drum)

    for cc in seq1.control_changes:
        instrument = cc.instrument if cc.instrument else 1
            
        result.control_changes.add(control_number=cc.control_number,
                                   control_value=cc.control_value,
                                   instrument=instrument,
                                   program=cc.program,
                                   is_drum=cc.is_drum)
        
    if seq2:
        for cc in seq2.control_changes:
            result.control_changes.add(control_number=cc.control_number,
                                       control_value=cc.control_value,
                                       instrument=cc.instrument + max_instrument_seq1,
                                       program=cc.program,
                                       is_drum=cc.is_drum)

    for ii in seq1.instrument_infos:
        instrument = ii.instrument if ii.instrument else 1
        
        result.instrument_infos.add(name=ii.name,
                                    instrument=instrument)
    if seq2:
        for ii in seq2.instrument_infos:
            result.instrument_infos.add(name=ii.name,
                                        instrument=ii.instrument + max_instrument_seq1)       

    result.source_info.encoding_type = seq1.source_info.encoding_type
    result.source_info.parser = seq1.source_info.parser
    result.total_time = seq1.total_time

    return result

# Продовження одноголосної мелодії

Імпорт одноголосного музичного файлу "Щедрик":

In [ ]:
schedrik_mono = note_seq.midi_file_to_note_sequence("schedrik_mono.mid")
play(schedrik_mono)

Завантаження моделі:

In [ ]:
note_seq.notebook_utils.download_bundle("lookback_rnn.mag", "models/")

Ініціалізація моделі:

In [ ]:
lookback_bundle = sequence_generator_bundle.read_bundle_file("models/lookback_rnn.mag")
lookback_generator_map = melody_rnn_sequence_generator.get_generator_map()
lookback_rnn = lookback_generator_map['lookback_rnn'](checkpoint=None, bundle=lookback_bundle)
lookback_rnn.initialize()

Продовження мелодії:

In [ ]:
schedrik_mono_continued = compose(schedrik_mono, lookback_rnn, 0.5)
play(schedrik_mono_continued)

# Продовження багатоголосної мелодії

Імпорт багатоголосного музичного файлу "Щедрик":

In [ ]:
schedrik_poly = note_seq.midi_file_to_note_sequence("schedrik_poly.mid")
play(schedrik_poly)

Завантаження моделі:

In [ ]:
note_seq.notebook_utils.download_bundle("polyphony_rnn.mag", "models/")

Ініціалізація моделі:

In [ ]:
polyphony_bundle = sequence_generator_bundle.read_bundle_file("models/polyphony_rnn.mag")
polyphony_generator_map = polyphony_sequence_generator.get_generator_map()
polyphony_rnn = polyphony_generator_map["polyphony"](checkpoint=None, bundle=polyphony_bundle)
polyphony_rnn.initialize()

Продовження мелодії:

In [ ]:
schedrik_poly_continued = compose(schedrik_poly, polyphony_rnn, 0.7)
play(schedrik_poly_continued)

# Сучасна композиція

Імпорт вступу відомої пісні:

In [ ]:
famous = note_seq.midi_file_to_note_sequence("famous.mid")
play(famous)

Виокремлена основна мелодія:

In [ ]:
famous_theme = note_seq.midi_file_to_note_sequence("famous_theme.mid")
play(famous_theme)

Підготовка акомпанементу:

In [ ]:
famous_backing = note_seq.midi_file_to_note_sequence("famous_backing.mid")
famous_backing_doubled = repeat_sequence_to_duration(famous_backing, famous_backing.total_time * 2)

Святославе Івановвичу, пробачте:

In [ ]:
famous_candidate = compose(famous_theme, lookback_rnn, 0.8)

with_backing = merge_from(famous_candidate, famous_backing_doubled)
play(with_backing)

# Про мене

![Про мене](images/about_me.png)